# Using GPT3 for event detection

My hope is that it might outperform the gelectra model by Wiedemann et al. 2022.

In [1]:
from src.data.protests.detection import load_glpn_dataset

glpn = load_glpn_dataset()
len(glpn["test"]), len(glpn["test.time"]), len(glpn["test.loc"])

Using custom data configuration default-552cac500ccb144a
Found cached dataset csv (/Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ceafcab5597f7c17.arrow
Loading cached processed dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-41bc297fe33d9da3.arrow
Loading cached processed dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-201012f13e291afc.arrow
Loading cached processed dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-b939ffa2b84d33a0.arrow
Loading cached processed dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317

(547, 752, 485)

In [2]:
import re
from time import sleep

import pandas as pd
from openai.error import InvalidRequestError
from tqdm.notebook import tqdm

from src.util.gpt import _query, query


def ja(str):
    return not not re.match(
        r".*(\W|^)ja(\W|$)", str, re.IGNORECASE | re.DOTALL | re.MULTILINE
    )


def nein(str):
    return not not re.match(
        r".*(\W|^)nein(\W|$)", str, re.IGNORECASE | re.DOTALL | re.MULTILINE
    )


def query_gpt(excerpt, prompt_func):
    # reduce excerpt size to roughly the max tokens of GPT3
    # this only affects a handful of excerpts
    max_len = int(4096 * 4 * 0.5)
    if len(excerpt) > max_len:
        print("Excerpt too long. Truncating. 😐")
        excerpt = excerpt[:max_len]
    prompt = prompt_func(excerpt)
    cost, response = query(prompt, max_tokens=500)
    response = response.replace("ja/nein", "")
    if ja(response) and nein(response):
        print("I'm confused. 🥺")
        print(response)
        label = 0
    if ja(response):
        label = 1
    elif nein(response):
        label = 0
    else:
        print("I'm insecure about my answer. 😢")
        print(response)
        label = 0
    return cost, response, label


def query_all(prompt_funcs, splits=["dev", "test", "test.time", "test.loc"], n=100):
    predictions = []
    for split in splits:
        print(split)
        cost = 0
        # only use a random sample of dataset split to save money
        articles = glpn[split].shuffle(seed=20230128).select(range(n))
        for article in tqdm(articles):
            for i, prompt_func in enumerate(prompt_funcs):
                cost_, response, label = query_gpt(article["excerpt"], prompt_func)
                cost += cost_
                predictions.append(
                    {
                        "split": split,
                        "prompt_type": i,
                        "predicted": label,
                        "reference": article["label"],
                        "excerpt": article["excerpt"],
                        "response": response,
                    }
                )
                # sleep(2)
        print(f"Cost for {split}: {cost}")
    return pd.DataFrame(predictions)

## First attempt

In [3]:
prompt1 = (
    lambda excerpt: f"{excerpt} \n\n Beschreibt dieser Zeitungsartikel ein Protestereignis? (Dazu zählen vielfältige Protestformen, wie Demonstrationen, Streiks, Blockaden, Unterschriftensammlungen, Besetzungen, Boykotte, etc.) Antworte mit ja oder nein.\n\n Antwort: "
)

In [4]:
predictions = query_all([prompt1])

Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0a0f4007f42216b4.arrow


dev


  0%|          | 0/100 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-98a701c0525cae92.arrow


Cost for dev: 1.3251400000000002
test


...

  0%|          | 0/100 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c152779ac42e62b1.arrow


Excerpt too long. Truncating. 😐
Cost for test.time: 1.0483000000000002
test.loc


  0%|          | 0/100 [00:00<?, ?it/s]

Cost for test.loc: 0.7838000000000002


In [5]:
import evaluate

f1 = evaluate.load("f1")


def evaluate(predictions):
    for split in predictions["split"].unique():
        for prompt_type in predictions["prompt_type"].unique():
            df_part = predictions[
                (predictions["split"] == split)
                & (predictions["prompt_type"] == prompt_type)
            ]
            print(f"Split: {split}, Prompt type: {prompt_type}")
            f1_score = f1.compute(
                predictions=list(df_part["predicted"]),
                references=list(df_part["reference"]),
            )
            print(f"F1: {f1_score}")

In [6]:
evaluate(predictions)

Split: dev, Prompt type: 0
F1: {'f1': 0.8080808080808082}
Split: test, Prompt type: 0
F1: {'f1': 0.8833333333333334}
Split: test.time, Prompt type: 0
F1: {'f1': 0.7666666666666666}
Split: test.loc, Prompt type: 0
F1: {'f1': 0.6486486486486486}


GPT3 with this prompt is generally worse than the finetuned gelectra-large model. Interesting is that even this method (which does not depend on finetuning) is worse on the test.time and test.loc sets. Maybe the gelectra-model _does_ generalize successfully, and the time and loc test splits are just inherently harder for some reason.

Here it is only evaluated on 100 examples; I had previously run it on the complete test data but somehow lost the cache for that; the results had been similar.

## Second attempt

I try it with another prompt, trying to apply both _role-playing_ (suggested on Twitter) and _chain of thought_ (suggested in a few papers that I'm too lazy to cite).

In [7]:
prompt2 = (
    lambda excerpt: f'Sie sind ein intelligenter und exakter PhD-Student in Politikwissenschaften. Bitte lesen Sie den folgenden Zeitungsartikel und entscheiden Sie dann, ob der Artikel ein Protestereignis beschreibt. Zu Protestereignissen zählen vielfältige Protestformen, wie Demonstrationen, Streiks, Blockaden, Unterschriftensammlungen, Besetzungen, Boykotte, etc. Bitte begründen Sie Ihre Antwort kurz.\n\n[Beginn des Zeitungsartikels]\n\n{excerpt}\n\n[Ende des Zeitungsartikels.]\n\nBeschreibt dieser Zeitungsartikel ein Protestereignis?\n\n(Feld 1: "Begründung", Feld 2: "Antwort")\n\n Begründung: '
)

In [8]:
predictions = query_all([prompt1, prompt2], splits=["dev"])

Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0a0f4007f42216b4.arrow


dev


  0%|          | 0/100 [00:00<?, ?it/s]

Cost for dev: 3.096039999999998


In [9]:
evaluate(predictions)

Split: dev, Prompt type: 0
F1: {'f1': 0.8080808080808082}
Split: dev, Prompt type: 1
F1: {'f1': 0.7964601769911505}


No significant improvement.

## Third attempt

I look at the model predictions to learn about the kind of mistakes that the model makes. I only look at the dev set. (Although it would be interesting to look at the test.loc set to see what's apparently harder there.)

In [10]:
n = 5
df = predictions.copy()
for i, row in list(df.iterrows())[:n]:
    if row["predicted"] != row["reference"]:
        df_part = df[(df["prompt_type"] == 1) & (df["excerpt"] == row["excerpt"])]
        print(row["reference"], row["predicted"])
        print(row["excerpt"])
        print()
        print(row["response"])
        print()

0 1
Stuttgarter Zeitung 2010-11-23 Abrissgegner für Schlichtung in Rheinfelden Von Wolfgang Messner  Die Abrissgegner geben nicht auf. Obwohl der Rückbau des alten Kraftwerks Rheinfelden seit Anfang November läuft, wollen sie alles tun, um die vollständige Zerstörung aufzuhalten. Die schweizerisch-deutsche Bürgerinitiative Pro Steg fordert nun eine Schlichtung wie im Falle von Stuttgart 21. An Stelle von Heiner Geißler soll Professor Karl Ganser als Verhandlungsführer engagiert werden. Bei dem handelt es sich um das älteste noch erhalten gebliebene Großwasserkraftwerk der Welt. Trotz internationaler Proteste von Denkmalschützern, Historikern und Architekten und einflussreicher Verbände soll es abgerissen werden. Ob einem Schlichtungsverfahren allerdings der für den Denkmalschutz zuständige Wirtschaftsminister Ernst Pfister (FDP) und die für die Ökologie verantwortliche Ressortchefin Tanja Gönner (CDU) zustimmen werden, bezweifeln Beobachter. Die Abrissgegner von der IG Steg sind über d

Based on the classification and reasoning results of the 2nd prompt on the dev set, I revise the prompt and evaluate it again on the dev set.

In [11]:
prompt3 = (
    lambda excerpt: f'Sie sind eine hochintelligente und exakte PhD-Studentin in Politikwissenschaften. Bitte lesen Sie den folgenden Zeitungsartikel und entscheiden Sie dann, ob der Artikel direkt über ein konkretes kürzlich stattgefundenes Protestereignis berichtet. Ein Protestereignis ist vor allem dadurch definiert, dass sich eine zivilgesellschaftliche Gruppe an einem konkreten Ort und Zeitpunkt außerhalb von etablierten politischer Strukturen (Regierung, Parlament, Parteien, Wirtschaftsverbände, etc.) mit einer politischen Botschaft an die Öffentlichkeit richtet. Der Begriff ist weit gefasst und geht von Plakaten und Unterschriftensammlungen über Demonstrationen und Kundgebungen bis zu gewaltsamen Ausschreitungen und Hasskriminalität (dies sind nur einige beispielhafte Kategorien). Bitte begründen Sie Ihre Antwort kurz, und geben Sie ggf. auch die kommunizierte politische Botschaft an.\n\n[Beginn des Zeitungsartikels]\n\n{excerpt}\n\n[Ende des Zeitungsartikels.]\n\nBeschreibt dieser Zeitungsartikel ein Protestereignis?\n\n(Feld 1: "Begründung", Feld 2: "Politische Botschaft", Feld 3: "Antwort (ja/nein)")\n\n Begründung: '
)

In [12]:
predictions = query_all([prompt1, prompt2, prompt3], splits=["dev"])

Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/csv/default-552cac500ccb144a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0a0f4007f42216b4.arrow


dev


  0%|          | 0/100 [00:00<?, ?it/s]

Cost for dev: 5.331120000000003


In [13]:
evaluate(predictions)

Split: dev, Prompt type: 0
F1: {'f1': 0.8080808080808082}
Split: dev, Prompt type: 1
F1: {'f1': 0.7964601769911505}
Split: dev, Prompt type: 2
F1: {'f1': 0.7678571428571429}


The revised prompt is not significantly better than the previous prompts.

GPT3 zero-shot classification is therefore no good alternative to the gelectra finetuning.

GPT3 finetuning might work but is very expensive for the davinci model, and the cheaper curie model is distinctly less intelligent.